In [2]:
import pandas as pd

file_path = 'credit.txt'
df = pd.read_csv(file_path, delimiter=',')
x = df.drop(columns='class').values.tolist()
y = df['class'].values.tolist()

In [3]:
class Node:
    def __init__(self, value=None):
        self.feature = None # feature index
        self.threshold = None # feature threshold
        self.value = value # feature index majority
        self.children = [] # child nodes

    def add(self, child):
        self.children.append(child)

    def remove(self, child):
        self.children.remove(child)

    def __str__(self):
        return str(self.value)


In [7]:
def split(x, y, feature, threshold):
    x1 = []
    x2 = []
    y1 = []
    y2 = []
    for i in range(len(x)):
        if x[i][feature] < threshold:
            x1.append(x[i])
            y1.append(y[i])
        else:
            x2.append(x[i])
            y2.append(y[i])
    return x1,x2,y1,y2

([[22, 0, 0, 28, 1], [24, 1, 1, 24, 1], [25, 0, 0, 27, 1], [29, 1, 1, 32, 0], [36, 1, 0, 52, 1], [23, 0, 1, 40, 0]], [[46, 0, 1, 32, 0], [45, 1, 1, 30, 0], [63, 1, 1, 58, 1], [50, 1, 1, 28, 0]], [0, 0, 0, 0, 1, 1], [0, 1, 1, 1])


In [10]:
for i in range(len(x)-1):
    print(x[i][0])

22
46
24
25
29
45
63
36
23
50


In [ ]:
import random
from collections import Counter

def pure(y):
    if len(set(y)) == 1: # if all the same classes in remaining 
        return True
    else:
        return False

def tree_grow(x, y, nmin, minleaf, nfeat):
    # nmin: if a node contains fewer cases than nmin, it becomes a leaf node.

    # minleaf: a split that creates a node with fewer than minleaf observations is not acceptable.
    #If there is no split that meets the minleaf constraint, the node becomes a leaf node. use GINI

    #nfeat we first draw at random nfeat features from which the best split is to be selected.

    while len(x) > 0:
        if pure(y): # if pure
            current_node.value = y[0]

        if len(y) < nmin: # if fewer cases than nmin
            current_node.value = max(y, key=y.count)

        ## do GINI

        best_split = split(x, best_feature, best_threshold) # x1,x2,y1,y2
        for i in range(2): # twice calling tree grow for each split
            parent = tree_grow(best_split[i], best_split[i+2], nmin, minleaf, nfeat)
        
    return root


    def tree_grow_b(x, y, nmin, minleaf, nfeat, m):
        return
    

    return
        

def tree_pred(x, tr):


    def tree_pred_b(trees, x):
        return
    

    return


root = Node()
current_node = root

tree_grow(x,y)